<a href="https://colab.research.google.com/github/7ev3r/Midterm_assignment_v2/blob/main/Train_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366457 sha256=b1eeb2b04cec589966f3542e0393c1f131dd679354fa23d39ac80c70ec41b670
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


Importing Libraries

In [98]:
import pandas as pd
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np
import time

In [100]:
df = pd.read_csv('Clean_whiskey_data.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.shape

(37567, 7)

In [101]:
pd.options.display.max_colwidth = 10000

In [102]:
df.head()

,whiskey,rating,url,User_ID,Whiskey_ID,price_euro,price_dollar
0,royal-lochnagar-2008,88,https://www.whiskybase.com/whiskies/whisky/224840/royal-lochnagar-2008,771,407,190.0,207.10
1,laphroaig-1967-rwd,100,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,609,248,65431.0,71319.79
2,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,636,248,65431.0,71319.79
3,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,766,248,65431.0,71319.79
4,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,642,248,65431.0,71319.79


In [103]:
df['whiskey'] = df['whiskey'].str.replace('-', ' ').str.title()

In [104]:
rec_df = df.drop(columns = ['url', 'price_dollar', 'price_euro'])
rec_df = rec_df[['User_ID', 'Whiskey_ID', 'rating']]

In [105]:
rec_match = df[['url', 'whiskey', 'Whiskey_ID', 'price_dollar', 'price_euro']].drop_duplicates('Whiskey_ID')

DataFrame with unique WhiskeyID's

In [106]:
rec_match.head()

,url,whiskey,Whiskey_ID,price_dollar,price_euro
0,https://www.whiskybase.com/whiskies/whisky/224840/royal-lochnagar-2008,Royal Lochnagar 2008,407,207.10,190.0
1,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,Laphroaig 1967 Rwd,248,71319.79,65431.0
38,https://www.whiskybase.com/whiskies/whisky/23249/laphroaig-1970-rwd,Laphroaig 1970 Rwd,250,9808.91,8999.0
55,https://www.whiskybase.com/whiskies/whisky/12078/strathisla-1967-rwd,Strathisla 1967 Rwd,452,32427.50,29750.0
77,https://www.whiskybase.com/whiskies/whisky/8854/bowmore-1966-sa,Bowmore 1966 Sa,78,81409.92,74688.0


In [107]:
rec_df.User_ID.max()

1008

Fitting model with data

In [129]:
from surprise import Reader, Dataset
reader = Reader(rating_scale = (1,100))
data = Dataset.load_from_df(rec_df,reader)

In [130]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.25, random_state = 100)

In [131]:
svd = SVD()
svd.fit(trainset)
preds = svd.test(testset)

Rating the model

In [132]:
accuracy.rmse(preds)

RMSE: 1.8064


1.806442045015489

Testing the model

In [133]:
def test_pal(uid, iid, actual):
    pred = svd.predict(uid, iid, actual, verbose=True)
    match = rec_match.loc[rec_match['Whiskey_ID'] == iid]
    return match

In [134]:
test_pal(609,2, 88)

user: 609        item: 2          r_ui = 88.00   est = 94.02   {'was_impossible': False}


,url,whiskey,Whiskey_ID,price_dollar,price_euro
6813,https://www.whiskybase.com/whiskies/whisky/15625/ardbeg-1959-ca,Ardbeg 1959 Ca,2,1853.0,1700.0


Function to return recommendations to the user

In [92]:
def get_recommendations(user_ratings, num_recs, priceEUR):
    new_ratings_df = rec_match.append(user_ratings,ignore_index=True)
    new_ratings_df = new_ratings_df[new_ratings_df['price_euro'] < priceEUR]
    new_ratings_df = new_ratings_df.drop(columns=['url', 'whiskey', 'price_dollar', 'price_euro'])

    #load in new df
    new_data = Dataset.load_from_df(new_ratings_df,reader)
    #create new svd object
    svd_new = SVD()
    #refit the model
    svd_new.fit(new_data.build_full_trainset())

    # predictions for the user
    list_of_whiskies = []
    for w_id in new_ratings_df['Whiskey_ID'].unique():
        list_of_whiskies.append((w_id, svd_new.predict(new_ratings_df['User_ID'].max(),w_id)[3]))

    # order the predictions from highest to lowest rated
    ranked_whiskies = sorted(list_of_whiskies, key=lambda x:x[1],reverse=True)

    rec_num = 1
    for i in ranked_whiskies[:num_recs]:
        recommended = rec_match[rec_match['Whiskey_ID'] == i[0]]
        print('Recommendation number:', rec_num)
        print('Whiskey: ' + recommended.values[0][1])
        print('Price: ' + str(recommended.values[0][-1]))
        print('URL: ' + recommended.values[0][0])
        print('\n')
        rec_num +=1



In [95]:
def BoozePal(df, num):
    userID = rec_df.User_ID.max()+1
    num_recs = input('Number of recommendations ? Enter a number from 1 to 10:\n')
    
    while int(num_recs) > 10:
        num_recs = input('You entered a number over 10. Please enter a number from 1 to 10 to continue. \n')
    priceEUR = input('Enter your budget for a bottle. EUR ')
    rating_list = []
    
    while num > 0:
        whiskey = df[df['price_euro'] < int(priceEUR)].sample(1)
        print('\nPlease rate the following {} whiskies. \n'.format(num))
        print('Whiskey: ' + whiskey.values[0][2])
        print('Price: ' + str(whiskey.values[0][-1]))
        print('URL: ' + str(whiskey.values[0][1]))
        rating = input('Rate this whiskey on a scale of 1-100, press n if you do not know it. :\n')
            
        if rating == 'n':
            continue
        
        if int(rating) > 100:
            print('Rating must be below 100!')
            continue
        
        else:
            rating_one_whiskey = {'User_ID':userID,'Whiskey_ID': whiskey['Whiskey_ID'].values[0],'rating': rating}
            rating_list.append(rating_one_whiskey) 
            num -= 1
        time.sleep(.5)
    print('\n'+'BoozePal recommendation : '+'\n')
    
    time.sleep(1.5)
    get_recommendations(rating_list, int(num_recs), int(priceEUR))

In [97]:
user_ratings= BoozePal(df, 5)

Number of recommendations ? Enter a number from 1 to 10:
3
Enter your budget for a bottle. EUR 100

Please rate the following 5 whiskies. 

Whiskey: https://www.whiskybase.com/whiskies/whisky/74716/bowmore-1972
Price: 97.01
URL: 91
Rate this whiskey on a scale of 1-100, press n if you do not know it. :
100

Please rate the following 4 whiskies. 

Whiskey: https://www.whiskybase.com/whiskies/whisky/95396/port-ellen-1969-gm
Price: 0.0
URL: 95
Rate this whiskey on a scale of 1-100, press n if you do not know it. :
10

Please rate the following 3 whiskies. 

Whiskey: https://www.whiskybase.com/whiskies/whisky/85425/glenglassaugh-37-year-old
Price: 0.0
URL: 94
Rate this whiskey on a scale of 1-100, press n if you do not know it. :
50

Please rate the following 2 whiskies. 

Whiskey: https://www.whiskybase.com/whiskies/whisky/85423/glenglassaugh-44-year-old
Price: 97.01
URL: 94
Rate this whiskey on a scale of 1-100, press n if you do not know it. :
100

Please rate the following 1 whiskies. 